In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

: 

In [ ]:
df = pd.read_excel('/home/dev/Projets/Gestion-des-analyses-de-variation/analyseVariation/v.xlsx')
df = df.dropna()
copie=df.copy()
data = df.copy()
d = df.copy()

: 

##### Dataset avec les valeurs nulles supprimées

In [ ]:
df.head(5)

: 

#### Récuperer la liste des noms

In [ ]:
l = list(df['Nom'])
l

: 

#### Faire une fonction qui va convertir tous les chaines en entier 

In [ ]:
# Faire une fonction qui extrait les chaines de caractères et les convertissent en entier
mesures = list(df['Mesures'].dropna())

def replace_comma_with_dot(lst):
    """Replace commas with dots in all elements of a list."""
    new_lst = []
    for element in lst:
        if isinstance(element, str) and ',' in element:
            element = element.replace(',', '.').split('.')[0]
        new_lst.append(int(element))
    return new_lst

: 

#### Démo

In [ ]:
replace_comma_with_dot(mesures)

: 

#### Construire un nouveau dataset avec les deux listes 

In [ ]:
# Construire un nouveau dataset
def dataset(list1,liste2):
    data = list(zip(list1,replace_comma_with_dot(liste2)))
    new_df = pd.DataFrame(data, columns=['Nom', 'Mesures'])
    return new_df

: 

In [ ]:
r = dataset(l,mesures)
r

: 

In [ ]:
r['Mesures'] = pd.to_numeric(r['Mesures'], errors='coerce')
limite_ctrl_sup = round(r['Mesures'].std() + r['Mesures'].mean(), 2)
limite_ctrl_sup

: 

In [ ]:
limite_ctrl_inf = round(r['Mesures'].mean() - r['Mesures'].std(), 2)
limite_ctrl_inf

: 

In [ ]:
nbre_va_sous_perf = r[r["Mesures"]<limite_ctrl_inf].Nom.count()
nbre_va_sous_perf

: 

In [ ]:
nbre_va_sur_perf = r[r["Mesures"]> limite_ctrl_sup].Nom.count()
nbre_va_sous_perf

: 

In [ ]:
r[(r["Mesures"]<650)|(r['Mesures']> 1000)]

: 

#### Fonctions pour visualization des valeurs aberrantes

In [ ]:
def boxplot(data):
    plt.figure(figsize = (16,6))
    sns.boxplot(data = data)
    plt.grid()

: 

In [ ]:
boxplot(r['Mesures'])

: 

#### Valeurs Non-aberrantes

In [ ]:
# result = [x for x in new_df['Mesures'] if new_df[x] > 550 and new_df[x] < 1000]
# print(result)



: 

#### Les valeurs aberrantes

In [ ]:
features = ['Mesures']

: 

In [ ]:
def impute_outliers(df, feature):
    q1 = np.percentile(df[feature], 25)
    q3 = np.percentile(df[feature], 75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5*iqr
    upper_bound = q3 + 1.5*iqr
    
    df.loc[df[feature] < lower_bound, feature] = lower_bound
    df.loc[df[feature] > upper_bound, feature] = upper_bound

: 

#### Enlever les valeurs aberrantes en applicant la fonction

In [ ]:
for feature in features:
    # print(feature)
    impute_outliers(r, feature)

: 

In [ ]:
boxplot(r[features])

: 

##### Un sous dataset contenant les valeurs aberrantes

In [ ]:
def subdataset_outliers(df, feature):
    q1 = np.percentile(df[feature], 25)
    q3 = np.percentile(df[feature], 75) 
    iqr = q3 - q1
    lower_bound = q1 - 1.5*iqr
    upper_bound = q3 + 1.5*iqr
    
    return df[(df[feature] < lower_bound) | (df[feature] > upper_bound)]

: 

In [ ]:
for feature in features:
    # print(feature)
    subdataset_outlier =subdataset_outliers(r, feature)

: 

In [ ]:
subdataset_outlier[features].max()

: 

In [ ]:
subdataset_outlier[features].shape

: 

#### Préptraiter les données

In [ ]:
def preprocess_data(data):
    # Remplacer les valeurs manquantes par la médiane de la colonne
    data = data.fillna(data.median())
    # Eliminer les observations ayant des valeurs manquantes
    data = data.dropna()
    return data

: 

In [ ]:
p = preprocess_data(d)
p

: 

In [ ]:
mesure = list(p['Mesures'])
nom = list(p['Nom'])

: 

In [ ]:
nouveau = dataset(nom,mesure)
nouveau

: 

In [ ]:
boxplot(nouveau['Mesures'])

: 

In [ ]:
def detect_outliers(data, threshold=3):
    # Calculer la moyenne et l'écart-type pour chaque colonne
    mean = data.mean(axis=0)
    std = data.std(axis=0)

    # Calculer le score Z pour chaque observation
    z_scores = np.abs((data-mean)/std)

    # Identifier les observations qui ont un score Z supérieur au seuil
    is_outlier = z_scores > threshold

    # Séparer les observations en deux groupes: valeurs aberrantes et valeurs non aberrantes
    outliers = data[is_outlier.any(axis=1)]
    non_outliers = data[~is_outlier.any(axis=1)]

    return outliers, non_outliers

: 

In [ ]:
detect_outliers(nouveau)

: 

In [ ]:
subdataset_outlier[features]

: 

#### Détecter les valeurs Aberrantes

In [ ]:
import numpy as np

def detect_outliers(data, threshold=3):
    """
    Cette fonction prend en entrée un tableau NumPy et renvoie un tableau 
    contenant toutes les observations qui sont considérées comme des valeurs aberrantes
    en utilisant la méthode de l'écart-type.
    """
    outliers = []
    mean = np.mean(data)
    std = np.std(data)
    for value in data:
        z_score = (value-mean)/std
        if np.abs(z_score)<threshold:
            outliers.append(value)
    return outliers

# detect_outliers(feature)

: 

# recuperer les VA de copie dans un sous dataset

In [ ]:
mesure = list(copie['Mesures'])
nom = list(copie['Nom'])

: 

In [ ]:
dataset = dataset(nom,mesure)

: 

In [ ]:
def subdataset_outliers(df, feature):
    q1 = np.percentile(df[feature], 25)
    q3 = np.percentile(df[feature], 75) 
    iqr = q3 - q1
    lower_bound = q1 - 1.5*iqr
    upper_bound = q3 + 1.5*iqr
    
    return df[(df[feature] < lower_bound) | (df[feature] > upper_bound)]

: 

In [ ]:
d = ['Mesures']

: 

In [ ]:
for i in d:
    print(i)
    subdataset_outlier = subdataset_outliers(dataset,i)

: 

In [ ]:
subdataset_outlier[d]

: 

# Fin de recuperation

In [ ]:
def detect_outliers(data, threshold=3):
    # Calculer la moyenne et l'écart-type pour chaque colonne
    mean = data.mean(axis=0)
    std = data.std(axis=0)

    # Calculer le score Z pour chaque observation
    z_scores = np.abs((data-mean)/std)

    # Identifier les observations qui ont un score Z supérieur au seuil
    is_outlier = z_scores > threshold

    # Séparer les observations en deux groupes: valeurs aberrantes et valeurs non aberrantes
    outliers = data[is_outlier.any(axis=1)]
    non_outliers = data[~is_outlier.any(axis=1)]

    return non_outliers

: 

In [ ]:
# detect_outliers(copie)

: 

In [ ]:
DFrame = pd.read_csv('/home/dev/Projets/Gestion-des-analyses-de-variation/analyseVariation/supermarket_sales - Sheet1 - .csv')

: 

In [ ]:
DFrame.head(4)

: 

In [ ]:
DFrame.columns

: 

In [ ]:
DFrame.value_counts

: 

In [ ]:
def boxplot(data):
    plt.figure(figsize = (16,6))
    sns.boxplot(data = data)
    plt.grid()

: 

In [ ]:
boxplot(DFrame)

: 

In [ ]:
detect_outliers(DFrame)

: 

In [ ]:
python -m pip install jupyter notebook -U

: 

In [ ]:
import logging
import os

# Créer le dossier des logs s'il n'existe pas
if not os.path.exists('logs'):
    os.makedirs('logs')

# Initialiser le logging
logging.basicConfig(
    filename='logs/app.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(user)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Exemple d'utilisation du logging
user = "Alice"
logging.info("L'utilisateur a effectué une action", extra={"user": user})

# Lire les logs pour un utilisateur spécifique
with open('logs/app.log', 'r') as f:
    lines = f.readlines()

for line in lines:
    if user in line:
        print(line.strip())


: 